# 卷积神经网络之LeNet
这里是卷积神经网络一章的第二节，上一节介绍了CNN中的基础知识。从这一节开始用代码实现常见的卷积神经网络，第一个要介绍的是LeNet神经网络。

**卷积网络是如何解决全连接网络参数过大的问题的：**

卷积层尝试解决这两个问题。一方面，卷积层保留输入形状，使图像的像素在高和宽两个方向上的相关性均可能被有效识别；另一方面，卷积层通过滑动窗口将同一卷积核与不同位置的输入重复计算，从而避免参数尺寸过大。

LeNet这个名字来源于LeNet论文的第一作者Yann LeCun。LeNet展示了通过梯度下降训练卷积神经网络可以达到手写数字识别在当时最先进的结果。这个奠基性的工作第一次将卷积神经网络推上舞台，为世人所知。LeNet的网络结构如下图所示。
![](http://tangshusen.me/Dive-into-DL-PyTorch/img/chapter05/5.5_lenet.png)
## LeNet模型
LeNet分为卷积层块和全连接层块两个部分。下面我们分别介绍这两个模块。

卷积层块里的基本单位是**卷积层后接最大池化层**：卷积层用来识别图像里的空间模式，如**线条和物体局部**，之后的最大池化层则用来降低卷积层对位置的敏感性。卷积层块由两个这样的基本单位重复堆叠构成。在卷积层块中，每个卷积层都使用$5\times 5$的窗口，并在输出上使用sigmoid激活函数。第一个卷积层输出通道数为6，第二个卷积层输出通道数则增加到16。这是因为第二个卷积层比第一个卷积层的输入的高和宽要小，所以增加输出通道使两个卷积层的参数尺寸类似。卷积层块的两个最大池化层的窗口形状均为$2\times 2$，且步幅为2。由于池化窗口与步幅形状相同，池化窗口在输入上每次滑动所覆盖的区域互不重叠。

卷积层块的输出形状为(批量大小, 通道, 高, 宽)。当卷积层块的输出传入全连接层块时，全连接层块会将小批量中每个样本**变平（flatten）**。也就是说，全连接层的输入形状将变成二维，其中第一维是小批量中的样本，第二维是每个样本变平后的向量表示，且向量长度为通道、高和宽的乘积。全连接层块含3个全连接层。它们的输出个数分别是120、84和10，其中10为输出的类别个数。

下面我们通过`Sequential`类来实现LeNet模型。

In [17]:
import time
import torch as t
from torch import nn,optim 
import torchvision
import torchvision.transforms as transforms
import os
import struct
import numpy as np
# torch.optim是一个实现了各种优化算法的库。大部分常用的方法得到支持，
# 并且接口具备足够的通用性，使得未来能够集成更加复杂的方法。

In [18]:

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet,self).__init__()
        self.conv=nn.Sequential(
            nn.Conv2d(1,6,5), # in_channels, out_channels, kernel_size
            nn.Sigmoid(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(6,16,5),
            nn.Sigmoid(),
            nn.MaxPool2d(2,2),
        )
        self.fc=nn.Sequential(
            nn.Linear(16*4*4,120),# 这里为什么是4*4，按照图片应该是5*5才对，应该是输入图片换尺寸了
            nn.Sigmoid(),
            nn.Linear(120,84),
            nn.Sigmoid(),
            nn.Linear(84,10)
        )
    def forward(self,img):
        feature= self.conv(img)
        output=self.fc(feature.view(img.shape[0],-1))
        return output
# 还记得Sequential类吗？
#Sequential类可以接收一个子模块的有序字典（OrderedDict）
#或者一系列子模块作为参数来逐一添加Module的实例

In [19]:
net=LeNet()
print(net)

LeNet(
  (conv): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=256, out_features=120, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [27]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

In [28]:
X_train,y_train=load_mnist('../../fashion_mnist_data',kind='train')
X_test, y_test=load_mnist('../../fashion_mnist_data',kind='t10k')

In [29]:
print("shape:",X_train.shape)
print('Rows: %d, columns: %d' % (X_train.shape[0], X_train.shape[1]))

shape: (60000, 784)
Rows: 60000, columns: 784


In [38]:

img_rows,img_cols=28,28
x_train=X_train
x_test=X_test

if 'channels_first'== 'channels_first':
    x_train=x_train.reshape(x_train.shape[0],1,img_rows,img_rows)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
num_samples=x_train.shape[0]
print("num_samples:",num_samples)

x_train shape: (60000, 1, 28, 28)
60000 train samples
10000 test samples
num_samples: 60000


In [39]:
x_train_t=t.from_numpy(x_train)
net(x_train_t)

tensor([[ 0.3098,  0.1266,  0.1270,  ..., -0.6087, -0.8793, -0.1773],
        [ 0.3098,  0.1266,  0.1269,  ..., -0.6086, -0.8791, -0.1773],
        [ 0.3099,  0.1267,  0.1271,  ..., -0.6086, -0.8792, -0.1772],
        ...,
        [ 0.3099,  0.1266,  0.1270,  ..., -0.6086, -0.8792, -0.1772],
        [ 0.3099,  0.1267,  0.1271,  ..., -0.6086, -0.8792, -0.1772],
        [ 0.3099,  0.1267,  0.1271,  ..., -0.6088, -0.8793, -0.1771]],
       grad_fn=<AddmmBackward>)

In [34]:
lr, num_epochs = 0.001, 5
optimizer = t.optim.Adam(net.parameters(), lr=lr)
#train_iter=

In [ ]:
def train(net,train,test,num_epochs):
    print("training ON")
    loss=t.nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, batch_count, start = 0.0, 0.0, 0, 0, time.time()
        for 

In [44]:
mnist_train = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=True, download=True, transform=transforms.ToTensor())
mnist_test = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=False, download=True, transform=transforms.ToTensor())

Using downloaded and verified file: C:\Users\LFP/Datasets/FashionMNIST\FashionMNIST\raw\train-images-idx3-ubyte.gz
Extracting C:\Users\LFP/Datasets/FashionMNIST\FashionMNIST\raw\train-images-idx3-ubyte.gz to C:\Users\LFP/Datasets/FashionMNIST\FashionMNIST\raw
Using downloaded and verified file: C:\Users\LFP/Datasets/FashionMNIST\FashionMNIST\raw\train-labels-idx1-ubyte.gz
Extracting C:\Users\LFP/Datasets/FashionMNIST\FashionMNIST\raw\train-labels-idx1-ubyte.gz to C:\Users\LFP/Datasets/FashionMNIST\FashionMNIST\raw
Using downloaded and verified file: C:\Users\LFP/Datasets/FashionMNIST\FashionMNIST\raw\t10k-images-idx3-ubyte.gz
Extracting C:\Users\LFP/Datasets/FashionMNIST\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to C:\Users\LFP/Datasets/FashionMNIST\FashionMNIST\raw
Using downloaded and verified file: C:\Users\LFP/Datasets/FashionMNIST\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz
Extracting C:\Users\LFP/Datasets/FashionMNIST\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to C:\Users\LFP/Dat

NameError: name 'd2l' is not defined